https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri

*MLP

In [2]:

import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import numpy as np
import time
import copy
from torch.utils.data import TensorDataset
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
from scipy.ndimage import rotate as scipyrotate
from torchvision import datasets, transforms
import random
import matplotlib.pyplot as plt
import time
from torch.utils.data import Dataset

# Define your network architecture (MLP)
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size*4)
        self.fc2 = nn.Linear(hidden_size*4, hidden_size*2)
        self.fc3 = nn.Linear(hidden_size*2, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        y = F.relu(self.fc3(x))
        z = self.fc4(y)
        return z
    

def get_time():
    return str(time.strftime("[%Y-%m-%d %H:%M:%S]", time.localtime()))


train_data_dir = 'Training'

mean = [0.4914, 0.4822, 0.4465]
std = [0.2023, 0.1994, 0.2010]
transform = transforms.Compose([
                                transforms.Resize((224, 224)),  
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                #  transforms.Normalize(mean=mean, std=std)
                                 ])

train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

test_data_dir = 'Testing'
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
sample_img=next(iter(train_loader))[0][0]
im_size = (sample_img.shape[1], sample_img.shape[2])
channel = 3
num_classes =max(train_dataset.targets)+1
epochs = 40

In [3]:

import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import numpy as np
import time
import copy
from torch.utils.data import TensorDataset
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
from scipy.ndimage import rotate as scipyrotate
from torchvision import datasets, transforms
import random
import matplotlib.pyplot as plt
import time
from captum.attr import IntegratedGradients
from torch.utils.data import Dataset

# Define your network architecture (MLP)
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size*4)
        self.fc2 = nn.Linear(hidden_size*4, hidden_size*2)
        self.fc3 = nn.Linear(hidden_size*2, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        y = F.relu(self.fc3(x))
        z = self.fc4(y)
        return z
    

def get_time():
    return str(time.strftime("[%Y-%m-%d %H:%M:%S]", time.localtime()))


train_data_dir = 'Training'

mean = [0.4914, 0.4822, 0.4465]
std = [0.2023, 0.1994, 0.2010]
transform = transforms.Compose([
                                transforms.Resize((224, 224)),  
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                #  transforms.Normalize(mean=mean, std=std)
                                 ])

train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

test_data_dir = 'Testing'
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
sample_img=next(iter(train_loader))[0][0]
im_size = (sample_img.shape[1], sample_img.shape[2])
channel = 3
num_classes =max(train_dataset.targets)+1
epochs = 40
#------------------Train the Net--------------------------------
net= MLP(input_size=channel * im_size[0] * im_size[1], hidden_size=128*2, output_size=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(), lr=1e-3)

# Teacher training before starting the dataset distillation process
for epochy in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        output = net(data) 
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epochy==0:
        print(get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', loss.item())
    elif (epochy+1)%20==0:
        print(get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', loss.item())

del loss
del output


# make all parameters non-trainable, so as to make image_syn the only trainable parameter
for param in list(net.parameters()):
    param.requires_grad = False
#---------------------------------------------------------------------------
net.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'>> Accuracy of Model trained on Training set on training set: {(100 * correct / total):.2f}%')

# test the accuracy of the model on train_dataloader
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'>> Accuracy of Model trained on Training set on testing set: {(100 * correct / total):.2f}%')

[2023-08-28 15:23:19] Epoch:  1 Loss of model:  2.3245182037353516
[2023-08-28 15:27:38] Epoch:  20 Loss of model:  0.23126275837421417
[2023-08-28 15:32:11] Epoch:  40 Loss of model:  0.19747164845466614
>> Accuracy of Model trained on Training set on training set: 88.85%
>> Accuracy of Model trained on Training set on testing set: 71.07%


In [4]:
# count the number of parameters of net
print("========MLP Architecture=========")
for name, param in net.named_parameters():
    print(name, param.numel())


========MLP Architecture=========
fc1.weight 154140672
fc1.bias 1024
fc2.weight 524288
fc2.bias 512
fc3.weight 131072
fc3.bias 256
fc4.weight 1024
fc4.bias 4


# CNN

In [4]:
import optuna
from vit_pytorch import ViT
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import numpy as np
import time
import copy
from torch.utils.data import TensorDataset
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
from scipy.ndimage import rotate as scipyrotate
from torchvision import datasets, transforms
import random
import matplotlib.pyplot as plt
import time
from captum.attr import IntegratedGradients
from torch.utils.data import Dataset
from torch.utils.data import Subset, DataLoader
import os


def get_time():
    return str(time.strftime("[%Y-%m-%d %H:%M:%S]", time.localtime()))



class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        # Convolutional layer with 16 output channels, 3x3 kernel, and padding of 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        # Max pooling layer with 2x2 kernel and stride of 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layer with 16 * 112 * 112 input features and 128 output features
        self.fc1 = nn.Linear(16 * 112 * 112, 128)
        # Fully connected layer with 128 input features and 'num_classes' output features
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Apply convolutional layer, ReLU activation, and max pooling
        x = self.pool(F.relu(self.conv1(x)))
        # Reshape the feature maps for the fully connected layer
        x = x.view(-1, 16 * 112 * 112)
        # Apply the first fully connected layer and ReLU activation
        x = F.relu(self.fc1(x))
        # Apply the final fully connected layer without an activation function
        x = self.fc2(x)
        return x




im_size = (224, 224)  # Specify the image size

num_classes = 4  # Change this according to the number of classes in your dataset
num_headers=2
net = CNN(
    num_classes=num_classes,
)

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
train_data_dir = os.path.abspath("./Training")
transform = transforms.Compose([
                                transforms.Resize((224, 224)),  
                                transforms.ToTensor(),
                                # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                    transforms.Normalize(mean=mean, std=std)
                                ])

train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)


trainloader= torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_data_dir = os.path.abspath("./Testing")
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)
valloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)


net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

# Teacher training before starting the dataset distillation process
epochs=50
for epochy in range(epochs):
    running_loss=0.0
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        output = net(data) 
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epochy+1)%10==0:
        print('\t',get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', running_loss/len(trainloader))

net.eval()
for param in list(net.parameters()):
    param.requires_grad = False


del loss
del data
del output
torch.cuda.empty_cache()

# test the accuracy of the model on train_dataloader
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valloader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    val_acc=100 * correct / total
    print(f'\t>> Accuracy of Model trained on Training set on testing set: {val_acc:.2f}%')



del net
del outputs
del images
torch.cuda.empty_cache()


	 [2023-08-30 13:43:02] Epoch:  10 Loss of model:  0.0018491453872734888
	 [2023-08-30 13:46:03] Epoch:  20 Loss of model:  0.000392454970925529
	 [2023-08-30 13:49:10] Epoch:  30 Loss of model:  0.00015091459653275604
	 [2023-08-30 13:51:59] Epoch:  40 Loss of model:  7.135632214259305e-05
	 [2023-08-30 13:54:55] Epoch:  50 Loss of model:  3.732560847614271e-05
	>> Accuracy of Model trained on Training set on testing set: 76.14%


# Vision Transformer

Original vision transformer from authors

In [5]:
import optuna
from vit_pytorch import ViT
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import numpy as np
import time
import copy
from torch.utils.data import TensorDataset
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
from scipy.ndimage import rotate as scipyrotate
from torchvision import datasets, transforms
import random
import matplotlib.pyplot as plt
import time
from captum.attr import IntegratedGradients
from torch.utils.data import Dataset
from torch.utils.data import Subset, DataLoader
import os


def get_time():
    return str(time.strftime("[%Y-%m-%d %H:%M:%S]", time.localtime()))



#============== Hyperparamer tuning for ViT =================
# Define the training function
def objective(trial):
    im_size = (224, 224)  # Specify the image size

    num_classes = 4  # Change this according to the number of classes in your dataset
    num_headers=2
    net = ViT(
        image_size=im_size[0],
        patch_size=56,
        num_classes=num_classes,
        dim=trial.suggest_categorical("dim", [2 ** i for i in range(5, 7)]),
        depth=trial.suggest_categorical("depth", [2*i for i in range(1,3)]),
        heads= trial.suggest_categorical("num_heads", [num_headers*i for i in range(1, 3)]),
        mlp_dim=trial.suggest_categorical("mlp_dim", [2 ** i for i in range(3, 9)]),
        dropout=trial.suggest_loguniform("drop_out", 0.01, 0.5),
        emb_dropout=0.0,
        pool='cls'
    )

    device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    net.to(device)

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    train_data_dir = os.path.abspath("./Training")
    transform = transforms.Compose([
                                    transforms.Resize((224, 224)),  
                                    transforms.ToTensor(),
                                    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                     transforms.Normalize(mean=mean, std=std)
                                    ])

    train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)



    # Create a subset of the dataset with only 10 samples per class
    N_p_c=100
    subset_indices = []
    for class_idx in range(len(train_dataset.classes)):
        class_indices = [idx for idx, (_, label) in enumerate(train_dataset.imgs) if label == class_idx]
        subset_indices.extend(class_indices[:N_p_c])  # Sample 10 images per class

    train_subset = Subset(train_dataset, subset_indices)
    trainloader = DataLoader(train_subset, batch_size=64, shuffle=True)

    # trainloader= torch.utils.data.DataLoader(train_dataset, batch_size=128*2, shuffle=True)

    test_data_dir = os.path.abspath("./Testing")
    test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)
    valloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)


    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=trial.suggest_loguniform("lr", 1e-4, 1e-1))

    # Teacher training before starting the dataset distillation process
    epochs=30
    for epochy in range(epochs):
        running_loss=0.0
        for batch_idx, (data, target) in enumerate(trainloader):
            data, target = data.to(device), target.to(device)
            output = net(data) 
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        if (epochy+1)%10==0:
            print('\t',get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', running_loss/len(trainloader))

    net.eval()
    for param in list(net.parameters()):
        param.requires_grad = False


    del loss
    del data
    del output
    torch.cuda.empty_cache()

    # test the accuracy of the model on train_dataloader
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valloader:
            images=images.to(device)
            labels=labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_acc=100 * correct / total
        print(f'\t>> Accuracy of Model trained on Training set on testing set: {val_acc:.2f}%')



    del net
    del outputs
    del images
    torch.cuda.empty_cache()

    return val_acc



# Create a study object and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40)  # You can adjust the number of trials

# Print the best trial
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2023-08-29 12:14:45,518] A new study created in memory with name: no-name-02304979-4b86-483d-8e39-adb85dd8df6f


	 [2023-08-29 12:15:03] Epoch:  10 Loss of model:  0.8910382049424308
	 [2023-08-29 12:15:21] Epoch:  20 Loss of model:  1.163801908493042
	 [2023-08-29 12:15:40] Epoch:  30 Loss of model:  1.0422886184283666


[I 2023-08-29 12:15:41,316] Trial 0 finished with value: 26.395939086294415 and parameters: {'dim': 32, 'depth': 4, 'num_heads': 2, 'mlp_dim': 8, 'drop_out': 0.38680173398955103, 'lr': 0.013962932579972016}. Best is trial 0 with value: 26.395939086294415.


	>> Accuracy of Model trained on Training set on testing set: 26.40%
	 [2023-08-29 12:15:59] Epoch:  10 Loss of model:  0.11754028818437032
	 [2023-08-29 12:16:18] Epoch:  20 Loss of model:  0.31887421597327503
	 [2023-08-29 12:16:36] Epoch:  30 Loss of model:  0.007627190356808049


[I 2023-08-29 12:16:37,329] Trial 1 finished with value: 38.578680203045685 and parameters: {'dim': 64, 'depth': 4, 'num_heads': 4, 'mlp_dim': 16, 'drop_out': 0.08206483425286712, 'lr': 0.0024497735294532895}. Best is trial 1 with value: 38.578680203045685.


...

	>> Accuracy of Model trained on Training set on testing set: 41.88%
	 [2023-08-29 12:49:45] Epoch:  10 Loss of model:  0.17633504952703202
	 [2023-08-29 12:50:03] Epoch:  20 Loss of model:  0.02522521944982665
	 [2023-08-29 12:50:21] Epoch:  30 Loss of model:  0.01215252439890589


[I 2023-08-29 12:50:23,055] Trial 38 finished with value: 41.6243654822335 and parameters: {'dim': 64, 'depth': 4, 'num_heads': 4, 'mlp_dim': 32, 'drop_out': 0.012045294472843568, 'lr': 0.0002929181658528998}. Best is trial 20 with value: 46.192893401015226.


	>> Accuracy of Model trained on Training set on testing set: 41.62%
	 [2023-08-29 12:50:41] Epoch:  10 Loss of model:  0.7521351235253471
	 [2023-08-29 12:50:59] Epoch:  20 Loss of model:  0.1675166870866503
	 [2023-08-29 12:51:16] Epoch:  30 Loss of model:  0.07664497941732407


[I 2023-08-29 12:51:18,233] Trial 39 finished with value: 44.16243654822335 and parameters: {'dim': 32, 'depth': 2, 'num_heads': 2, 'mlp_dim': 128, 'drop_out': 0.018463816773958135, 'lr': 0.0001619395223903985}. Best is trial 20 with value: 46.192893401015226.


	>> Accuracy of Model trained on Training set on testing set: 44.16%
Number of finished trials:  40
Best trial:
Value:  46.192893401015226
Params: 
    dim: 32
    depth: 2
    num_heads: 2
    mlp_dim: 32
    drop_out: 0.019124017956274296
    lr: 0.00030051845934512427


In [6]:
im_size = (224, 224)  # Specify the image size

num_classes = 4  # Change this according to the number of classes in your dataset
num_headers=2
net = ViT(
    image_size=im_size[0],
    patch_size=56,
    num_classes=num_classes,
    dim=trial.params["dim"],
    depth=trial.params["depth"],
    heads=trial.params["num_heads"],
    mlp_dim=trial.params["mlp_dim"],
    dropout=trial.params["drop_out"],
    emb_dropout=0.0,
    pool='cls'
)

device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
net.to(device)

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
train_data_dir = os.path.abspath("./Training")
transform = transforms.Compose([
                                transforms.Resize((224, 224)),  
                                transforms.ToTensor(),
                                # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                    transforms.Normalize(mean=mean, std=std)
                                ])

train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)


trainloader= torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_data_dir = os.path.abspath("./Testing")
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)
valloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)


net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=trial.params["lr"])

# Teacher training before starting the dataset distillation process
epochs=30
for epochy in range(epochs):
    running_loss=0.0
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        output = net(data) 
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epochy+1)%10==0:
        print('\t',get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', running_loss/len(trainloader))

net.eval()
for param in list(net.parameters()):
    param.requires_grad = False


del loss
del data
del output
torch.cuda.empty_cache()

# test the accuracy of the model on train_dataloader
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valloader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    val_acc=100 * correct / total
    print(f'\t>> Accuracy of Model trained on Training set on testing set: {val_acc:.2f}%')



del net
del outputs
del images
torch.cuda.empty_cache()

	 [2023-08-29 13:00:22] Epoch:  10 Loss of model:  0.2734017180899779
	 [2023-08-29 13:02:32] Epoch:  20 Loss of model:  0.08274168616367711
	 [2023-08-29 13:04:43] Epoch:  30 Loss of model:  0.07401238263377713
	>> Accuracy of Model trained on Training set on testing set: 72.59%


# Simple Vision Transformer

An update from some of the same authors of the original paper proposes simplifications to ViT that allows it to train faster and better.

Among these simplifications include 2d sinusoidal positional embedding, global average pooling (no CLS token), no dropout, batch sizes of 1024 rather than 4096, and use of RandAugment and MixUp augmentations. They also show that a simple linear at the end is not significantly worse than the original MLP head

In [3]:
from vit_pytorch import SimpleViT

epochs= 80

simple_vision_net = SimpleViT(
    image_size = im_size[0],
    patch_size = 32,
    num_classes = num_classes,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048
)



simple_vision_net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(simple_vision_net.parameters(), lr=1e-3)

# Teacher training before starting the dataset distillation process
for epochy in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        output = simple_vision_net(data) 
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epochy==0:
        print(get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', loss.item())
    elif (epochy+1)%20==0:
        print(get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', loss.item())

del loss
del output


# make all parameters non-trainable, so as to make image_syn the only trainable parameter
for param in list(simple_vision_net.parameters()):
    param.requires_grad = False
#---------------------------------------------------------------------------
simple_vision_net.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = simple_vision_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'>> Accuracy of Model trained on Training set on training set: {(100 * correct / total):.2f}%')

# test the accuracy of the model on train_dataloader
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = simple_vision_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'>> Accuracy of Model trained on Training set on testing set: {(100 * correct / total):.2f}%')




[2023-12-03 13:12:02] Epoch:  1 Loss of model:  1.3234524726867676
[2023-12-03 13:16:18] Epoch:  20 Loss of model:  0.7982242107391357
[2023-12-03 13:20:50] Epoch:  40 Loss of model:  0.6601513028144836
[2023-12-03 13:25:53] Epoch:  60 Loss of model:  0.34663447737693787
[2023-12-03 13:30:51] Epoch:  80 Loss of model:  0.13117243349552155
>> Accuracy of Model trained on Training set on training set: 94.63%
>> Accuracy of Model trained on Training set on testing set: 64.21%


In [7]:
# count the number of parameters of vision_net
print("========Simplified Vision Transformer Architecture=========")
for name, param in simple_vision_net.named_parameters():
    print(name, param.numel())


========Simplified Vision Transformer Architecture=========
to_patch_embedding.1.weight 3072
to_patch_embedding.1.bias 3072
to_patch_embedding.2.weight 3145728
to_patch_embedding.2.bias 1024
to_patch_embedding.3.weight 1024
to_patch_embedding.3.bias 1024
transformer.norm.weight 1024
transformer.norm.bias 1024
transformer.layers.0.0.norm.weight 1024
transformer.layers.0.0.norm.bias 1024
transformer.layers.0.0.to_qkv.weight 3145728
transformer.layers.0.0.to_out.weight 1048576
transformer.layers.0.1.net.0.weight 1024
transformer.layers.0.1.net.0.bias 1024
transformer.layers.0.1.net.1.weight 2097152
transformer.layers.0.1.net.1.bias 2048
transformer.layers.0.1.net.3.weight 2097152
transformer.layers.0.1.net.3.bias 1024
transformer.layers.1.0.norm.weight 1024
transformer.layers.1.0.norm.bias 1024
transformer.layers.1.0.to_qkv.weight 3145728
transformer.layers.1.0.to_out.weight 1048576
transformer.layers.1.1.net.0.weight 1024
transformer.layers.1.1.net.0.bias 1024
transformer.layers.1.1.net.

In [53]:
del vision_net

# NaViT
This paper proposes to leverage the flexibility of attention and masking for variable lengthed sequences to train images of multiple resolution, packed into a single batch. They demonstrate much faster training and improved accuracies, with the only cost being extra complexity in the architecture and dataloading. They use factorized 2d positional encodings, token dropping, as well as query-key normalization.

In [10]:
from vit_pytorch.na_vit import NaViT
import torch
import torch
import torch.nn as nn
import numpy as np
import time
import copy
from torch.utils.data import TensorDataset
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
from scipy.ndimage import rotate as scipyrotate
from torchvision import datasets, transforms
import random
import matplotlib.pyplot as plt
import time
from captum.attr import IntegratedGradients
from torch.utils.data import Dataset


epochs=80
device = "cuda:0" if torch.cuda.is_available() else "cpu"

train_data_dir = 'Training'

mean = [0.4914, 0.4822, 0.4465]
std = [0.2023, 0.1994, 0.2010]
transform = transforms.Compose([
                                transforms.Resize((224, 224)),  
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                #  transforms.Normalize(mean=mean, std=std)
                                 ])
train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_data_dir = 'Testing'
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

sample_img=next(iter(train_loader))[0][0]
im_size = (sample_img.shape[1], sample_img.shape[2])

channel = 3

num_classes =max(train_dataset.targets)+1



na_vit = NaViT(
    image_size = im_size[0],
    patch_size = 32,
    num_classes = num_classes,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1,
    token_dropout_prob = 0.1  # token dropout of 10% (keep 90% of tokens)
)


def get_time():
    return str(time.strftime("[%Y-%m-%d %H:%M:%S]", time.localtime()))


na_vit.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(na_vit.parameters(), lr=1e-3)

# Teacher training before starting the dataset distillation process
for epochy in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # convert data to list along first dimension
        data = [list(data)]
        output = na_vit(data) 
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epochy==0:
        print(get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', loss.item())
    elif (epochy+1)%20==0:
        print(get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', loss.item())

del loss
del output


# make all parameters non-trainable, so as to make image_syn the only trainable parameter
for param in list(na_vit.parameters()):
    param.requires_grad = False
#---------------------------------------------------------------------------



[2023-08-24 17:49:00] Epoch:  1 Loss of model:  1.3943312168121338
[2023-08-24 17:54:23] Epoch:  20 Loss of model:  1.3738586902618408
[2023-08-24 18:00:18] Epoch:  40 Loss of model:  1.3188676834106445
[2023-08-24 18:06:09] Epoch:  60 Loss of model:  1.3631240129470825
[2023-08-24 18:12:00] Epoch:  80 Loss of model:  1.3580167293548584


NameError: name 'vision_net' is not defined

In [11]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images=images.to(device)
        labels=labels.to(device)
        images = [list(images)]
        outputs = na_vit(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'>> Accuracy of Model trained on Training set on training set: {(100 * correct / total):.2f}%')

# test the accuracy of the model on train_dataloader
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        images = [list(images)]
        outputs = na_vit(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'>> Accuracy of Model trained on Training set on testing set: {(100 * correct / total):.2f}%')


>> Accuracy of Model trained on Training set on training set: 28.64%
>> Accuracy of Model trained on Training set on testing set: 29.19%


In [12]:
# count the number of parameters of vision_net
print("========Na Vision Transformer Architecture=========")
for name, param in na_vit.named_parameters():
    print(name, param.numel())

========Na Vision Transformer Architecture=========
pos_embed_height 7168
pos_embed_width 7168
attn_pool_queries 1024
to_patch_embedding.0.gamma 3072
to_patch_embedding.1.weight 3145728
to_patch_embedding.1.bias 1024
to_patch_embedding.2.gamma 1024
transformer.layers.0.0.norm.gamma 1024
transformer.layers.0.0.q_norm.gamma 1024
transformer.layers.0.0.k_norm.gamma 1024
transformer.layers.0.0.to_q.weight 1048576
transformer.layers.0.0.to_kv.weight 2097152
transformer.layers.0.0.to_out.0.weight 1048576
transformer.layers.0.1.0.gamma 1024
transformer.layers.0.1.1.weight 2097152
transformer.layers.0.1.1.bias 2048
transformer.layers.0.1.4.weight 2097152
transformer.layers.0.1.4.bias 1024
transformer.layers.1.0.norm.gamma 1024
transformer.layers.1.0.q_norm.gamma 1024
transformer.layers.1.0.k_norm.gamma 1024
transformer.layers.1.0.to_q.weight 1048576
transformer.layers.1.0.to_kv.weight 2097152
transformer.layers.1.0.to_out.0.weight 1048576
transformer.layers.1.1.0.gamma 1024
transformer.layers.

# Pretrained DeiT

In [5]:
deit_model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)

# Update the model's final classifier for 4 classes
num_classes = 4
deit_model.head = torch.nn.Linear(deit_model.head.in_features, num_classes)

deit_model.to(device)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize(256, interpolation=3),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])


train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
trainloader= torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(deit_model.parameters(), lr=1e-5)

# Teacher training before starting the dataset distillation process
epochs=30
for epochy in range(epochs):
    running_loss=0.0
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        output = deit_model(data) 
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epochy+1)%10==0:
        print('\t',get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', running_loss/len(trainloader))




Using cache found in C:\Users\AMI_LAB/.cache\torch\hub\facebookresearch_deit_main


	 [2023-08-30 14:08:01] Epoch:  10 Loss of model:  0.007767439981841523
	 [2023-08-30 14:16:11] Epoch:  20 Loss of model:  0.0014806994611559355
	 [2023-08-30 14:24:23] Epoch:  30 Loss of model:  0.0006409694223016825


In [6]:
deit_model.eval()
for param in list(deit_model.parameters()):
    param.requires_grad = False


# test the accuracy of the model on train_dataloader
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valloader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = deit_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    val_acc=100 * correct / total
    print(f'\t>> Accuracy of Model trained on Training set on testing set: {val_acc:.2f}%')



# del net
del outputs
del images
torch.cuda.empty_cache()

	>> Accuracy of Model trained on Training set on testing set: 71.07%


In [16]:
# count the number of parameters of vision_net
print("========Data Efficient Vision Transformer Architecture=========")
for name, param in deit_model.named_parameters():
    print(name, param.numel())

========Data Efficient Vision Transformer Architecture=========
cls_token 768
pos_embed 151296
patch_embed.proj.weight 589824
patch_embed.proj.bias 768
blocks.0.norm1.weight 768
blocks.0.norm1.bias 768
blocks.0.attn.qkv.weight 1769472
blocks.0.attn.qkv.bias 2304
blocks.0.attn.proj.weight 589824
blocks.0.attn.proj.bias 768
blocks.0.norm2.weight 768
blocks.0.norm2.bias 768
blocks.0.mlp.fc1.weight 2359296
blocks.0.mlp.fc1.bias 3072
blocks.0.mlp.fc2.weight 2359296
blocks.0.mlp.fc2.bias 768
blocks.1.norm1.weight 768
blocks.1.norm1.bias 768
blocks.1.attn.qkv.weight 1769472
blocks.1.attn.qkv.bias 2304
blocks.1.attn.proj.weight 589824
blocks.1.attn.proj.bias 768
blocks.1.norm2.weight 768
blocks.1.norm2.bias 768
blocks.1.mlp.fc1.weight 2359296
blocks.1.mlp.fc1.bias 3072
blocks.1.mlp.fc2.weight 2359296
blocks.1.mlp.fc2.bias 768
blocks.2.norm1.weight 768
blocks.2.norm1.bias 768
blocks.2.attn.qkv.weight 1769472
blocks.2.attn.qkv.bias 2304
blocks.2.attn.proj.weight 589824
blocks.2.attn.proj.bias 76

In [7]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to C:\Users\AMI_LAB/.cache\torch\hub\checkpoints\efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:02<00:00, 8.50MB/s]

Loaded pretrained weights for efficientnet-b0


In [9]:
# count the number of parameters of vision_net
print("========Data Efficient Vision Transformer Architecture=========")
for name, param in model.named_parameters():
    print(name, param.numel())

========Data Efficient Vision Transformer Architecture=========
_conv_stem.weight 864
_bn0.weight 32
_bn0.bias 32
_blocks.0._depthwise_conv.weight 288
_blocks.0._bn1.weight 32
_blocks.0._bn1.bias 32
_blocks.0._se_reduce.weight 256
_blocks.0._se_reduce.bias 8
_blocks.0._se_expand.weight 256
_blocks.0._se_expand.bias 32
_blocks.0._project_conv.weight 512
_blocks.0._bn2.weight 16
_blocks.0._bn2.bias 16
_blocks.1._expand_conv.weight 1536
_blocks.1._bn0.weight 96
_blocks.1._bn0.bias 96
_blocks.1._depthwise_conv.weight 864
_blocks.1._bn1.weight 96
_blocks.1._bn1.bias 96
_blocks.1._se_reduce.weight 384
_blocks.1._se_reduce.bias 4
_blocks.1._se_expand.weight 384
_blocks.1._se_expand.bias 96
_blocks.1._project_conv.weight 2304
_blocks.1._bn2.weight 24
_blocks.1._bn2.bias 24
_blocks.2._expand_conv.weight 3456
_blocks.2._bn0.weight 144
_blocks.2._bn0.bias 144
_blocks.2._depthwise_conv.weight 1296
_blocks.2._bn1.weight 144
_blocks.2._bn1.bias 144
_blocks.2._se_reduce.weight 864
_blocks.2._se_reduc

In [16]:

# Update the model's final classifier for 4 classes
num_classes = 4

for param in model.parameters():
    param.requires_grad = True



class ModifiedModel(nn.Module):
    def __init__(self, original_model):
        super(ModifiedModel, self).__init__()
        # Create a new classifier head with output size 4
        self.featurizer= original_model
        self.classifier = nn.Sequential(
            nn.Linear(1000, 128),  # Adjust input size accordingly
            nn.ReLU(),
            nn.Linear(128, 4)  # Output size of 4 for 4 classes
        )

    def forward(self, x):
        # Use the modified classifier for forward pass
        x= self.featurizer(x)
        x = self.classifier(x)
        return x


efficient_net_model = ModifiedModel(model).to(device)

deit_model.to(device)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize(256, interpolation=3),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])


train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
trainloader= torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(efficient_net_model.parameters(), lr=1e-5)

# Teacher training before starting the dataset distillation process
epochs=60
for epochy in range(epochs):
    running_loss=0.0
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        output = efficient_net_model (data) 
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epochy+1)%10==0:
        print('\t',get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', running_loss/len(trainloader))




	 [2023-08-30 15:10:16] Epoch:  10 Loss of model:  0.11445411359486372
	 [2023-08-30 15:14:24] Epoch:  20 Loss of model:  0.04076709282463011
	 [2023-08-30 15:18:31] Epoch:  30 Loss of model:  0.02178373228272666
	 [2023-08-30 15:22:38] Epoch:  40 Loss of model:  0.010593964951112866
	 [2023-08-30 15:26:46] Epoch:  50 Loss of model:  0.00936040855721449
	 [2023-08-30 15:30:53] Epoch:  60 Loss of model:  0.010032777391050173


In [18]:
deit_model.eval()
for param in list(efficient_net_model.parameters()):
    param.requires_grad = False


# test the accuracy of the model on train_dataloader
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valloader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = efficient_net_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    val_acc=100 * correct / total
    print(f'\t>> Accuracy of Model trained on Training set on testing set: {val_acc:.2f}%')



# del net
del outputs
del images
torch.cuda.empty_cache()

	>> Accuracy of Model trained on Training set on testing set: 67.26%


In [22]:
# https://github.com/d-li14/efficientnetv2.pytorch/blob/main/effnetv2.py
import effnetv2 as bacha
train_data_dir = os.path.abspath("./Training")
test_data_dir = os.path.abspath("./Testing")
model = bacha.effnetv2_s(num_classes=4).to(device)

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize(256, interpolation=3),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])


train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
trainloader= torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


# Teacher training before starting the dataset distillation process
epochs=30
for epochy in range(epochs):
    running_loss=0.0
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        output = model(data) 
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epochy+1)%10==0:
        print('\t',get_time(), 'Epoch: ', epochy+1 , 'Loss of model: ', running_loss/len(trainloader))

model.eval()
for param in list(model.parameters()):
    param.requires_grad = False


del loss
del data
del output
torch.cuda.empty_cache()

# test the accuracy of the model on train_dataloader
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valloader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    val_acc=100 * correct / total
    print(f'\t>> Accuracy of Model trained on Training set on testing set: {val_acc:.2f}%')



# count the number of parameters of vision_net
print("========Data Efficient Vision Transformer Architecture=========")
for name, param in model.named_parameters():
    print(name, param.numel())

	 [2023-08-30 16:24:21] Epoch:  10 Loss of model:  0.6544434583705404
	 [2023-08-30 16:29:57] Epoch:  20 Loss of model:  0.08491142482861229
	 [2023-08-30 16:35:33] Epoch:  30 Loss of model:  0.0354600147060726
	>> Accuracy of Model trained on Training set on testing set: 37.56%
========Data Efficient Vision Transformer Architecture=========
features.0.0.weight 648
features.0.1.weight 24
features.0.1.bias 24
features.1.conv.0.weight 5184
features.1.conv.1.weight 24
features.1.conv.1.bias 24
features.1.conv.3.weight 576
features.1.conv.4.weight 24
features.1.conv.4.bias 24
features.2.conv.0.weight 5184
features.2.conv.1.weight 24
features.2.conv.1.bias 24
features.2.conv.3.weight 576
features.2.conv.4.weight 24
features.2.conv.4.bias 24
features.3.conv.0.weight 20736
features.3.conv.1.weight 96
features.3.conv.1.bias 96
features.3.conv.3.weight 4608
features.3.conv.4.weight 48
features.3.conv.4.bias 48
features.4.conv.0.weight 82944
features.4.conv.1.weight 192
features.4.conv.1.bias 19